In [1]:
import pandas as pd
import numpy as np

# Read in data

In [2]:
keep_cols = ['Type', 'Site', 'tau', 'mo_mi']

# Read in Catalan et al. (2016) data and drop bioassay data
# and field data from lakes and reservoirs

data_fpath = r"data/catalan_2016_doc_data.xlsx"
cat_df_full = pd.read_excel(data_fpath, usecols="A:M")

cat_df_full.rename(
    {
        "WRT (years) ": "tau",
        "Degradation rate temperature corrected (d-1)": "k_per_day",
        "FieldII": "Method",
        "Type": "Full_type",
        "System10": "Type",
        "Latitude (d.d.)": 'lat',
        "Longitude (d.d.)": 'lon',
        "Cite": "Ref",
    },
    axis="columns",
    inplace=True,
)

# Sort by tau for plotting later
cat_df_full = cat_df_full.sort_values(by="tau")

# Derive variables (don't trust variables I derived in Excel from their data when playing around in case errors crept in)
cat_df_full["k"] = cat_df_full["k_per_day"] * 365
cat_df_full["mo_mi"] = np.exp(-cat_df_full["k"] * cat_df_full["tau"])

# Drop bioassay data
cat_df = cat_df_full.loc[cat_df_full["Method"] == "FieldModel", keep_cols+['Excluded from analysis']]

# Drop lake and reservoir data (instead use data from Evans et al. 2017)
cat_df = cat_df[~cat_df['Type'].isin(['Lake', 'Reservoir'])]

# For plotting later, get field data that Catalan et al. didn't include in stats, but did plot
ocean_df = cat_df.loc[cat_df["Excluded from analysis"] == 1, keep_cols].copy()

# Drop data that Catalan et al. excluded from their regression analyses, for comparability
# (of the field data, this is just oceanic data)
cat_df = cat_df.loc[cat_df["Excluded from analysis"] == 0, keep_cols].copy()

# Calculate site means where have multi-year (or maybe depth) data
duplicates = cat_df[cat_df.duplicated('Site')]
print("\nDuplicates in the 'Site' column; means calculated:")
print(duplicates)

# Group by 'Site' and calculate mean for numeric columns
cat_df = cat_df.groupby('Site').agg({'Type': 'first', 'tau': 'mean', 'mo_mi': 'mean'}).reset_index()

# Add "source" column
cat_df['Source'] = "C2016"

cat_df.tail()


Duplicates in the 'Site' column; means calculated:
        Type           Site       tau     mo_mi
260    River  Roaring brook  0.030137  0.968531
264    River  Roaring brook  0.032877  0.699084
265    River  Roaring brook  0.032877  0.402985
262    River  Roaring brook  0.038356  0.699275
263    River  Roaring brook  0.038356  0.369845
266  Estuary        Gironde  1.500000  0.536481


,Site,Type,tau,mo_mi,Source
21,Cheesapeake bay,Estuary,0.972603,0.500000,C2016
22,Gironde,Estuary,1.500000,0.712685,C2016
23,Hudson261-44 km,River,0.148356,0.912081,C2016
24,Kryckland,River,0.005479,0.997600,C2016
25,Roaring brook,River,0.031917,0.640500,C2016


In [10]:
# Quick look at the oceanic data and save for plotting later
ocean_df.to_csv(r'data/C2016_oceanic_data.csv', index=False)
ocean_df

,Type,Site,tau,mo_mi
331,Ocean,Arctic (Beaufort Gyre),12.5,0.297453
332,Ocean,Arctic,30.0,0.910448
329,Ocean,Greenland to North Atlantic,80.0,0.908333
328,Ocean,Nordic seas,128.0,0.976676
330,Ocean,Greenland to North Atlantic,595.0,0.814458


In [4]:
# Read in lake and reservoir data from Evans et al. (2017)
data_fpath = r"data/evans_2017_doc_retention_cleaned.csv"
ev_df = pd.read_csv(data_fpath)

ev_df.sort_values("tau", inplace=True)

# Drop any NANs in tau or RDOC
ev_df = ev_df.dropna(how="any", subset=["tau", "RDOC"])

print(ev_df.columns)

# Drop negative retention values
# Evans et al. defined net sink lakes as having RDOC > 0.1. I will use
# a more lenient definition, with a threshold of 0, as I model transmission in the range 0-1
ev_df = ev_df.query("RDOC > 0")

# Test sensitivity to two lakes with high tau
# ev_df = ev_df.query("tau < 100")

# Calculate transmission
ev_df["TDOC"] = 1 - ev_df["RDOC"]

# Rename for compatibility with other data
ev_df.rename(
    {
        "name": "Site",
        "type": "Type",
        "TDOC": "mo_mi",
    },
    axis="columns",
    inplace=True,
)

ev_df = ev_df[keep_cols]

# Add "source" column
ev_df['Source'] = 'E2017'

ev_df.head()

Index(['name', 'country', 'type', 'nutrient_status', 'tau', 'DOC_in',
       'DOC_out', 'TDOC', 'RDOC', 'Notes', 'DOC_type'],
      dtype='object')


,Type,Site,tau,mo_mi,Source
22,Lake,Ekojarvi,0.07,0.90,E2017
27,Lake,Røynelandsvatn,0.10,0.86,E2017
51,Lake,Lundetjenn,0.10,0.99,E2017
35,Reservoir,Black Esk,0.12,0.79,E2017
33,Lake,Gäddtjärn,0.13,0.78,E2017


In [5]:
# Read in extra riv data (reviewed by me)
riv_df = pd.read_csv(r'data/review_instream_doc_retention_summary.csv')
riv_df['tau'] = riv_df['t (days)'] / 365  # tau in years
riv_df['Type'] = 'River'
riv_df.rename({'Transmission (M_out/M_in)': 'mo_mi'},
              axis=1, inplace=True)

# Drop data associated with natural DOM addition experiments
riv_df = riv_df.loc[riv_df['Exclude_expts'] == 0]

riv_df = riv_df[keep_cols]

# Add source info
riv_df['Source'] = 'This study'

riv_df.head()

,Type,Site,tau,mo_mi,Source
0,River,"NE Spain, site and sample median",0.000685,1.000000,This study
1,River,"Wood Brook, England",0.000249,0.982609,This study
3,River,"Creeping Swamp, North Carolina, USA",0.025368,0.360000,This study
4,River,"Fort River, Massachusetts, USA",0.000363,0.960000,This study
5,River,"SE China, TLC, average",0.002426,0.713333,This study


# Combine data and save

In [6]:
# Concatenate the three dataframes
df = pd.concat([cat_df, ev_df, riv_df])

# Clean up some remaining messiness in the string columns
df['Type'] = df['Type'].str.strip().str.lower()
df['Type'] = df['Type'].replace('lake*', 'lake')

# Check for duplicates in the 'Site' column
duplicates = df[df.duplicated('Site', keep=False)]
if len(duplicates) > 0:
    # duplicates.to_csv('duplicates.csv')
    print("\nDuplicates in the 'Site' column:")
    print(duplicates)
else:
    print('No site duplicates')

# Save
df.to_csv(r'data/combined_data_2025-04.csv', index=False)

df

No site duplicates


,Site,Type,tau,mo_mi,Source
0,1,catchment,1.400000,0.630,C2016
1,2,catchment,0.500000,0.680,C2016
2,3,catchment,0.500000,0.690,C2016
3,4,catchment,0.500000,0.560,C2016
4,5,catchment,2.600000,0.530,C2016
...,...,...,...,...,...
7,"Glensaugh, Scotland",river,0.000114,0.770,This study
8,"Medium river, MA (USA)",river,0.013699,0.850,This study
9,"Peaty headwater, NE Scotland",river,0.000576,0.863,This study
11,"River Tees, UK",river,0.007991,0.415,This study


# Quick description of the data

In [7]:
print(df.describe())

              tau      mo_mi
count   92.000000  92.000000
mean    11.448968   0.612731
std     73.909100   0.209544
min      0.000114   0.100000
25%      0.245000   0.477500
50%      1.300000   0.590000
75%      2.512500   0.770000
max    692.000000   1.000000


In [8]:
stats_df = df.groupby('Type').describe()
stats_df.to_csv(r'data/compiled-data-stats.csv')

stats_df

tau                                                       \
          count       mean         std       min       25%       50%   
Type                                                                   
catchment  21.0   2.852381    3.089275  0.400000  1.000000  2.100000   
estuary     2.0   1.236301    0.372926  0.972603  1.104452  1.236301   
lake       49.0  20.168231  100.918047  0.070000  0.700000  1.660000   
reservoir   6.0   0.408333    0.443460  0.120000  0.192500  0.255000   
river      14.0   0.017083    0.039103  0.000114  0.000512  0.001934   

                                mo_mi                                      \
                75%         max count      mean       std   min       25%   
Type                                                                        
catchment  3.100000   13.500000  21.0  0.486190  0.132721  0.23  0.390000   
estuary    1.368151    1.500000   2.0  0.606342  0.150391  0.50  0.553171   
lake       2.700000  692.000000  49.0  0.599796  0.206756  0.10  0.490000   
reservoir  0.325000    1.300000   6.0  0.788333  0.158419  0.55  0.712500   
river      0.012272    0.148356  14.0  0.773469  0.206086  0.36  0.668333   

                                         
                50%       75%       max  
Type                                     
catchment  0.490000  0.560000  0.690000  
estuary    0.606342  0.659514  0.712685  
lake       0.580000  0.740000  0.990000  
reservoir  0.785000  0.917500  0.960000  
river      0.810000  0.948020  1.000000

In [9]:
df.groupby('Source').describe()

tau                                                      \
           count       mean        std       min       25%       50%   
Source                                                                 
C2016       26.0   2.406091   2.928672  0.005479  0.525000  1.750000   
E2017       55.0  18.012606  95.349440  0.070000  0.385000  1.590000   
This study  11.0   0.004855   0.008031  0.000114  0.000427  0.000685   

                                 mo_mi                                      \
                 75%         max count      mean       std   min       25%   
Source                                                                       
C2016       2.575000   13.500000  26.0  0.537418  0.179141  0.23  0.432500   
E2017       2.625000  692.000000  55.0  0.620364  0.209381  0.10  0.510000   
This study  0.005209    0.025368  11.0  0.752581  0.214523  0.36  0.682222   

                                     
              50%       75%     max  
Source                               
C2016       0.505  0.647625  0.9976  
E2017       0.590  0.780000  0.9900  
This study  0.770  0.911500  1.0000